In [2]:
import re
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,StratifiedKFold

In [3]:
df_sourceData = pd.read_csv('./广医-现病史.csv')
print(df_sourceData.info())
df_sourceData.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 36 columns):
现病史              2999 non-null object
经常咳嗽_稳定期         2999 non-null object
经常咳嗽时间           2999 non-null object
经常喘息_稳定期         2999 non-null object
经常喘息时间           2999 non-null object
经常胸闷_稳定期         2999 non-null object
经常胸闷时间           2999 non-null object
经常活动后气促_稳定期      2999 non-null object
经常活动后气促时间        2999 non-null object
经常咳痰_稳定期         2999 non-null object
经常咳痰时间           2999 non-null object
经常咳痰痰量           2999 non-null object
诱因_加重期           2999 non-null object
时间_加重期           2999 non-null object
咳嗽_加重期           2999 non-null object
咳嗽_时间            2999 non-null object
咳痰_加重期           2999 non-null object
咳痰_时间            2999 non-null object
胸闷_加重期           2999 non-null object
胸闷_时间            2999 non-null object
喘息_加重期           2999 non-null object
喘息_时间            2999 non-null object
气促_加重期           2999 non-null object
气促_时间            29

,现病史,经常咳嗽_稳定期,经常咳嗽时间,经常喘息_稳定期,经常喘息时间,经常胸闷_稳定期,经常胸闷时间,经常活动后气促_稳定期,经常活动后气促时间,经常咳痰_稳定期,...,发热_加重期,发热_时间,胸痛_加重期,胸痛_时间,心悸_加重期,心悸_时间,夜间阵发性呼吸困难_加重期,夜间阵发性呼吸困难_时间,双下肢水肿_加重期,双下肢水肿_时间
0,患者约10年余前无明显诱因出现活动后气促，伴少咳嗽，咳少量白色粘痰。无胸痛，无发热，无...,有,10年,未提及,未提及,无,无,有,10年,有,...,无,无,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及
1,患者10日前无明显诱因出现发热，体温最高至39.7度，伴咳嗽咳痰，痰为黄色粘痰，不易咳出，伴...,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,...,有,10天,有,10天,无,无,未提及,未提及,未提及,未提及
2,患者于7余年前无明显反复出现咳嗽、咳痰、气促、心悸，伴夜间阵发性呼吸困难，曾在当地医院就医，...,有,7年,未提及,未提及,未提及,未提及,未提及,未提及,有,...,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及
3,患者于１年余前活动后出现气促，平素步行至3楼便觉气促，休息后可以缓解，无头晕、头痛，无咳嗽、...,无,无,未提及,未提及,未提及,未提及,有,1年,无,...,无,无,未提及,未提及,未提及,未提及,有,7天,未提及,未提及
4,患儿于8年前无明显诱因出现咳嗽，非犬吠样咳，咳嗽多于季节转变时加重。6年前出现痰多，为黄色粘...,有,8年,未提及,未提及,未提及,未提及,未提及,未提及,有,...,有,5天,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及


In [4]:
df_sourceData.columns

Index(['现病史', '经常咳嗽_稳定期', '经常咳嗽时间', '经常喘息_稳定期', '经常喘息时间', '经常胸闷_稳定期', '经常胸闷时间',
       '经常活动后气促_稳定期', '经常活动后气促时间', '经常咳痰_稳定期', '经常咳痰时间', '经常咳痰痰量', '诱因_加重期',
       '时间_加重期', '咳嗽_加重期', '咳嗽_时间', '咳痰_加重期', '咳痰_时间', '胸闷_加重期', '胸闷_时间',
       '喘息_加重期', '喘息_时间', '气促_加重期', '气促_时间', '咯血_加重期', '咯血_时间', '发热_加重期',
       '发热_时间', '胸痛_加重期', '胸痛_时间', '心悸_加重期', '心悸_时间', '夜间阵发性呼吸困难_加重期',
       '夜间阵发性呼吸困难_时间', '双下肢水肿_加重期', '双下肢水肿_时间'],
      dtype='object')

In [53]:
df_sourceData = df_sourceData[['现病史','经常咳嗽_稳定期', '经常喘息_稳定期', '经常胸闷_稳定期',
       '经常活动后气促_稳定期', '经常咳痰_稳定期', '咳嗽_加重期', '咳痰_加重期', '胸闷_加重期',
       '喘息_加重期', '气促_加重期', '咯血_加重期', '发热_加重期', '胸痛_加重期', '心悸_加重期', 
        '夜间阵发性呼吸困难_加重期', '双下肢水肿_加重期']]
print(df_sourceData['现病史'].tolist()[:5])
df_sourceData.loc[:,'现病史'] = df_sourceData['现病史'].str.replace('\n','')
df_sourceData.loc[:,'现病史'] = df_sourceData['现病史'].str.replace(' ','')
df_sourceData.head()
print(df_sourceData['现病史'].tolist()[:5])

['   患者约10年余前无明显诱因出现活动后气促，伴少咳嗽，咳少量白色粘痰。无胸痛，无发热，无咯血、盗汗、消瘦。间有夜间阵发性呼吸困难，无心悸、胸闷，无双下肢浮肿，无少尿等。5年前于广州军区陆军总医院门诊就诊，拟“慢性阻塞性肺气肿”予“激素”治疗（具体诊治欠详），症状可稍缓解，但气促常反复，未经正规诊治。2年前，患者上述症状再发，气促加重，运动耐量进行性下降，遂来我院门诊就诊，拟“AECOPD”收入我科继续治疗，予激素抗炎、沐舒坦化痰、及氨茶碱和可必特平喘解痉等处理后好转出院。出院后规范用舒利迭等吸入试剂治疗。今年5月份患者因气促加重入住广州市陆军总医院加强治疗，好转后出院。1月前患者诉气促再次加重，活动后明显，咳嗽咳痰较前无明显加重。患者近一个月以来，无发热为，无咯血，无胸闷，无午后低热，胃纳、精神一般，二便正常，体重无明显增减。', '患者10日前无明显诱因出现发热，体温最高至39.7度，伴咳嗽咳痰，痰为黄色粘痰，不易咳出，伴右侧胸肋部胸痛，咳嗽时加重，伴气促，活动后加重，伴口干口苦，无畏寒、寒战，无胸闷心悸，无头晕头痛，无恶心呕吐。遂于清远市中医院治疗，当地CT示：考虑右下肺叶炎症并脓肿形成。遂予以退热、抗感染（用过头孢哌酮舒巴坦7天，亚胺培南3天）、化痰、支持等治疗后，患者仍然反复发热，无明显好转。今患者为求进一步诊治入我院求诊，门诊以“右肺炎并脓肿”收住我科。入院后患者精神、食欲、睡眠一般，小便无异常，可见褐色大便。起病来患者体重无明显增减。', '患者于7余年前无明显反复出现咳嗽、咳痰、气促、心悸，伴夜间阵发性呼吸困难，曾在当地医院就医，诊断为“冠心病 陈旧性心肌梗死 ”，经治疗（具体不详）好转后出院。2月前上述症状加重，休息时既有发作，夜间不能平卧，无双下肢浮肿，即到当地医院住院治疗，予以对症治疗后症状好转，但出院后上诉症状反复发作，伴夜间阵发性呼吸困难及端坐呼吸，体力活动明显受限，伴咳嗽，咳痰，为白色稀痰，无血丝痰，无双下肢浮肿。无畏寒、发热，无恶心、呕吐，见有胸痛、心悸，无黑朦、晕厥、抽搐，为求进一步诊治入我院就医，门诊以“冠心病”收入我科治疗。患者起病以来，饮食稍差，睡眠一般，大小便正常，体重无明显改变。', '患者于１年余前活动后出现气促，平素步行至3楼便觉气促，休息后可以缓解，无头晕、头痛，无咳嗽、咳痰，无恶心、呕吐,间中端坐呼吸，夜间阵发

In [54]:
list_text = df_sourceData['现病史'].tolist()
list_text = [x.replace("\n","") for x in list_text]
print(list_text[:5])

['患者约10年余前无明显诱因出现活动后气促，伴少咳嗽，咳少量白色粘痰。无胸痛，无发热，无咯血、盗汗、消瘦。间有夜间阵发性呼吸困难，无心悸、胸闷，无双下肢浮肿，无少尿等。5年前于广州军区陆军总医院门诊就诊，拟“慢性阻塞性肺气肿”予“激素”治疗（具体诊治欠详），症状可稍缓解，但气促常反复，未经正规诊治。2年前，患者上述症状再发，气促加重，运动耐量进行性下降，遂来我院门诊就诊，拟“AECOPD”收入我科继续治疗，予激素抗炎、沐舒坦化痰、及氨茶碱和可必特平喘解痉等处理后好转出院。出院后规范用舒利迭等吸入试剂治疗。今年5月份患者因气促加重入住广州市陆军总医院加强治疗，好转后出院。1月前患者诉气促再次加重，活动后明显，咳嗽咳痰较前无明显加重。患者近一个月以来，无发热为，无咯血，无胸闷，无午后低热，胃纳、精神一般，二便正常，体重无明显增减。', '患者10日前无明显诱因出现发热，体温最高至39.7度，伴咳嗽咳痰，痰为黄色粘痰，不易咳出，伴右侧胸肋部胸痛，咳嗽时加重，伴气促，活动后加重，伴口干口苦，无畏寒、寒战，无胸闷心悸，无头晕头痛，无恶心呕吐。遂于清远市中医院治疗，当地CT示：考虑右下肺叶炎症并脓肿形成。遂予以退热、抗感染（用过头孢哌酮舒巴坦7天，亚胺培南3天）、化痰、支持等治疗后，患者仍然反复发热，无明显好转。今患者为求进一步诊治入我院求诊，门诊以“右肺炎并脓肿”收住我科。入院后患者精神、食欲、睡眠一般，小便无异常，可见褐色大便。起病来患者体重无明显增减。', '患者于7余年前无明显反复出现咳嗽、咳痰、气促、心悸，伴夜间阵发性呼吸困难，曾在当地医院就医，诊断为“冠心病陈旧性心肌梗死”，经治疗（具体不详）好转后出院。2月前上述症状加重，休息时既有发作，夜间不能平卧，无双下肢浮肿，即到当地医院住院治疗，予以对症治疗后症状好转，但出院后上诉症状反复发作，伴夜间阵发性呼吸困难及端坐呼吸，体力活动明显受限，伴咳嗽，咳痰，为白色稀痰，无血丝痰，无双下肢浮肿。无畏寒、发热，无恶心、呕吐，见有胸痛、心悸，无黑朦、晕厥、抽搐，为求进一步诊治入我院就医，门诊以“冠心病”收入我科治疗。患者起病以来，饮食稍差，睡眠一般，大小便正常，体重无明显改变。', '患者于１年余前活动后出现气促，平素步行至3楼便觉气促，休息后可以缓解，无头晕、头痛，无咳嗽、咳痰，无恶心、呕吐,间中端坐呼吸，夜间阵发性呼吸困难

In [55]:
df_label = df_sourceData[['经常咳嗽_稳定期', '经常喘息_稳定期', '经常胸闷_稳定期',
       '经常活动后气促_稳定期', '经常咳痰_稳定期', '咳嗽_加重期', '咳痰_加重期', '胸闷_加重期',
       '喘息_加重期', '气促_加重期', '咯血_加重期', '发热_加重期', '胸痛_加重期', '心悸_加重期', 
        '夜间阵发性呼吸困难_加重期', '双下肢水肿_加重期']]
df_label.head()

,经常咳嗽_稳定期,经常喘息_稳定期,经常胸闷_稳定期,经常活动后气促_稳定期,经常咳痰_稳定期,咳嗽_加重期,咳痰_加重期,胸闷_加重期,喘息_加重期,气促_加重期,咯血_加重期,发热_加重期,胸痛_加重期,心悸_加重期,夜间阵发性呼吸困难_加重期,双下肢水肿_加重期
0,有,未提及,无,有,有,未提及,未提及,无,未提及,未提及,无,无,未提及,未提及,未提及,未提及
1,未提及,未提及,未提及,未提及,未提及,有,有,无,未提及,有,未提及,有,有,无,未提及,未提及
2,有,未提及,未提及,未提及,有,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及
3,无,未提及,未提及,有,无,有,有,未提及,未提及,有,未提及,无,未提及,未提及,有,未提及
4,有,未提及,未提及,未提及,有,有,有,未提及,无,无,有,有,未提及,未提及,未提及,未提及


In [56]:
list_nodeName = list(df_label.columns)
print(list_nodeName)

['经常咳嗽_稳定期', '经常喘息_稳定期', '经常胸闷_稳定期', '经常活动后气促_稳定期', '经常咳痰_稳定期', '咳嗽_加重期', '咳痰_加重期', '胸闷_加重期', '喘息_加重期', '气促_加重期', '咯血_加重期', '发热_加重期', '胸痛_加重期', '心悸_加重期', '夜间阵发性呼吸困难_加重期', '双下肢水肿_加重期']


In [57]:
list_nodeValue = ['未提及','有','无']

In [58]:
for i in range(len(list_nodeValue)):
    df_label.replace(list_nodeValue[i], i, inplace=True)
    
print(df_label.head())

   经常咳嗽_稳定期  经常喘息_稳定期  经常胸闷_稳定期  经常活动后气促_稳定期  经常咳痰_稳定期  咳嗽_加重期  咳痰_加重期  \
0         1         0         2            1         1       0       0   
1         0         0         0            0         0       1       1   
2         1         0         0            0         1       0       0   
3         2         0         0            1         2       1       1   
4         1         0         0            0         1       1       1   

   胸闷_加重期  喘息_加重期  气促_加重期  咯血_加重期  发热_加重期  胸痛_加重期  心悸_加重期  夜间阵发性呼吸困难_加重期  \
0       2       0       0       2       2       0       0              0   
1       2       0       1       0       1       1       2              0   
2       0       0       0       0       0       0       0              0   
3       0       0       1       0       2       0       0              1   
4       0       2       2       1       1       0       0              0   

   双下肢水肿_加重期  
0          0  
1          0  
2          0  
3          0  
4          0  


/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


# 处理文本到预训练格式

In [59]:
list_splitSentencesOfText = []
for i in range(len(list_text)):
    sentences = re.split('(。|！|\!|？|\?)',list_text[i])
    new_sents = []
    for i in range(int(len(sentences)/2)):
        sent = sentences[2*i] + sentences[2*i+1]
        new_sents.append(sent)
    list_splitSentencesOfText.append(new_sents)
    
print(list_splitSentencesOfText[:2])

[['患者约10年余前无明显诱因出现活动后气促，伴少咳嗽，咳少量白色粘痰。', '无胸痛，无发热，无咯血、盗汗、消瘦。', '间有夜间阵发性呼吸困难，无心悸、胸闷，无双下肢浮肿，无少尿等。', '5年前于广州军区陆军总医院门诊就诊，拟“慢性阻塞性肺气肿”予“激素”治疗（具体诊治欠详），症状可稍缓解，但气促常反复，未经正规诊治。', '2年前，患者上述症状再发，气促加重，运动耐量进行性下降，遂来我院门诊就诊，拟“AECOPD”收入我科继续治疗，予激素抗炎、沐舒坦化痰、及氨茶碱和可必特平喘解痉等处理后好转出院。', '出院后规范用舒利迭等吸入试剂治疗。', '今年5月份患者因气促加重入住广州市陆军总医院加强治疗，好转后出院。', '1月前患者诉气促再次加重，活动后明显，咳嗽咳痰较前无明显加重。', '患者近一个月以来，无发热为，无咯血，无胸闷，无午后低热，胃纳、精神一般，二便正常，体重无明显增减。'], ['患者10日前无明显诱因出现发热，体温最高至39.7度，伴咳嗽咳痰，痰为黄色粘痰，不易咳出，伴右侧胸肋部胸痛，咳嗽时加重，伴气促，活动后加重，伴口干口苦，无畏寒、寒战，无胸闷心悸，无头晕头痛，无恶心呕吐。', '遂于清远市中医院治疗，当地CT示：考虑右下肺叶炎症并脓肿形成。', '遂予以退热、抗感染（用过头孢哌酮舒巴坦7天，亚胺培南3天）、化痰、支持等治疗后，患者仍然反复发热，无明显好转。', '今患者为求进一步诊治入我院求诊，门诊以“右肺炎并脓肿”收住我科。', '入院后患者精神、食欲、睡眠一般，小便无异常，可见褐色大便。', '起病来患者体重无明显增减。']]


In [60]:
f = open('./medical_text.txt','w')
for i in  range(len(list_splitSentencesOfText)):
    l_t = list_splitSentencesOfText[i]
    for ii in range(len(l_t)):    
        f.write(l_t[ii])
        f.write('\n')
    f.write('\n')
    
f.close()

In [61]:
df_sourceData_train,df_sourceData_val,df_sourceData_test = df_sourceData[:1500],df_sourceData[1500:2000],df_sourceData[2000:2999]
print(len(df_sourceData_test))
print(df_sourceData_test.head())

999
                                                    现病史 经常咳嗽_稳定期 经常喘息_稳定期  \
2000  患者5年前无明显诱因出现咳嗽，偶有黄色浓痰，无血丝痰，多在冬季及气温寒冷时发作，每次都伴有气...        有      未提及   
2001  患者1年前于在郑州某医院体检时，行CT检查示：右下肺阴影。一直未予治疗，门诊随访。7日前来我...      未提及      未提及   
2002  患者2年前在无诱因下出现咳嗽，起初症状轻，以干咳为主，无咳痰，无明显发热、畏寒，无胸闷、气促...        有      未提及   
2003  患者于20余年前无明显诱因下反复出现咳嗽，阵发性连声咳，咳少许白色泡沫痰，晨起明显，无咳血，...        有      未提及   
2004  患者入院前14年，常无明显诱因出现咳嗽、咳痰，阵发性，咳少许白色粘液性痰，无咳浓痰及血丝痰，...        有      未提及   

     经常胸闷_稳定期 经常活动后气促_稳定期 经常咳痰_稳定期 咳嗽_加重期 咳痰_加重期 胸闷_加重期 喘息_加重期 气促_加重期 咯血_加重期  \
2000        无         未提及        有      有    未提及    未提及    未提及      有    未提及   
2001      未提及         未提及      未提及      无      无      无    未提及      无    未提及   
2002        无         未提及        无      有      有    未提及    未提及      有      无   
2003      未提及           有        有      有      有    未提及    未提及      有      无   
2004        无           有        有    未提及    未提及    未提及    未提及    未提及    未提及   

     发热_加重期 胸痛_加重期 心悸_加重期 夜间阵发性呼吸困难_加重期 双下肢水肿_加重期  


In [62]:
df_sourceData_train.to_csv('medical_train.tsv',sep='\t',encoding='utf8')
df_sourceData_val.to_csv('medical_val.tsv',sep='\t',encoding='utf8')
df_sourceData_test.to_csv('medical_test.tsv',sep='\t',encoding='utf8')